In [1]:
%load_ext autoreload
%autoreload 2

import tqdm, sys, os, time
import pandas as pd
import scipy as sp

import scanpy as sc
import anndata as ad

# Load scRNAseq data

In [3]:
adata = ad.read_h5ad("cache/data-raw-scRNAseq_10x_v3_AIBS-matrix.h5ad",).transpose()
adata

AnnData object with n_obs × n_vars = 176584 × 31053

## Load samples

In [83]:
obs = pd.read_csv("data/raw/scRNAseq_10x_v3_AIBS/barcodes.tsv.gz", 
                  compression='gzip', skiprows=1,
                  names=['sample_name'], index_col=0)


metadata = pd.read_csv("data/raw/scRNAseq_10x_v3_AIBS/sample_metadata.csv", 
                       dtype={'Region': 'category',
                              'Lib_type': 'category',
                              'Gender': 'category',
                              'Donor': 'str',
                             },
                       index_col=0)
metadata.index.name = 'sample_name'


cluster_membership = pd.read_csv("data/raw/scRNAseq_10x_v3_AIBS/cluster.membership.csv", 
                                 names=['sample_name', 'cluster_id'], 
                                 dtype={'cluster_id': 'category'},
                                 skiprows=1, index_col=0)

cluster_annotation = pd.read_csv("data/raw/scRNAseq_10x_v3_AIBS/cluster.annotation.csv", 
                                 index_col='cluster_id', dtype={'cluster_id': 'category'})

cluster_membership = cluster_membership.join(cluster_annotation, on='cluster_id')


obs = obs.join(metadata).join(cluster_membership)
obs.describe(percentiles=[])

,aggr_num,umi.counts,gene.counts,Amp_Date,Amp_PCR_cyles,Lib_Date,Lib_PCR_cycles,Cell_Capture,Lib_Cells,Mean_Reads_perCell,Median_Genes_perCell,Median_UMI_perCell,Total_Cells,Live_Cells,mapped_reads,unmapped_reads,nonconf_mapped_reads,total.reads,doublet.score,size
count,176584.000000,176584.000000,176584.000000,176584,176584.0,176584,176584.000000,176584.000000,176584.000000,176584.000000,176584.000000,176584.000000,176584.000000,176584.000000,1.765840e+05,176584.000000,176584.000000,1.765840e+05,176584.000000,94170.000000
mean,6.585851,30534.110389,4452.454039,2019-01-21 23:06:34.201060096,12.0,2019-01-28 16:45:42.608616704,10.403581,4666.475870,16700.022403,130999.594499,3774.248986,17492.789551,6816.154742,4586.634208,9.482265e+04,1943.099834,9282.957420,1.060487e+05,0.114185,6392.821451
min,1.000000,500.000000,69.000000,2018-11-29 00:00:00,12.0,2018-12-11 00:00:00,9.000000,2960.000000,8535.000000,85837.000000,728.000000,1044.000000,4289.000000,2466.000000,5.340000e+02,0.000000,13.000000,5.470000e+02,0.000000,11.000000
50%,7.000000,19362.000000,4889.000000,2018-12-07 00:00:00,12.0,2018-12-11 00:00:00,10.000000,5094.000000,16692.000000,116944.000000,3636.000000,10689.000000,7078.000000,5094.000000,5.863550e+04,1156.000000,6007.500000,6.591700e+04,0.066667,5854.000000
max,12.000000,336286.000000,12287.000000,2019-04-26 00:00:00,12.0,2019-04-30 00:00:00,11.000000,5898.000000,25378.000000,189925.000000,6946.000000,45605.000000,9598.000000,5898.000000,1.078829e+06,24399.000000,110114.000000,1.193540e+06,0.980000,17334.000000
std,3.385489,34412.951488,3318.534699,NaN,0.0,NaN,0.592404,929.288491,5570.952733,34875.649653,2488.091004,16251.133514,1616.140661,1081.268542,1.092397e+05,2338.474244,10286.237424,1.216521e+05,0.145514,6210.070362


In [84]:
adata.obs = obs

## Load variables

In [67]:
var = pd.read_table("data/raw/scRNAseq_10x_v3_AIBS/features.tsv.gz", 
                    compression='gzip', header=None, 
                    names=['gene_id', 'gene_name', 'type'], 
                    usecols=['gene_id', 'gene_name'])
adata.var = var

## AnnData 

In [85]:
adata

AnnData object with n_obs × n_vars = 176584 × 31053
    obs: 'aggr_num', 'umi.counts', 'gene.counts', 'library_id', 'tube_barcode', 'Seq_batch', 'Region', 'Lib_type', 'Gender', 'Donor', 'Amp_Name', 'Amp_Date', 'Amp_PCR_cyles', 'Lib_Name', 'Lib_Date', 'Replicate_Lib', 'Lib_PCR_cycles', 'Lib_PassFail', 'Cell_Capture', 'Lib_Cells', 'Mean_Reads_perCell', 'Median_Genes_perCell', 'Median_UMI_perCell', 'Saturation', 'Live_percent', 'Total_Cells', 'Live_Cells', 'method', 'exp_component_name', 'mapped_reads', 'unmapped_reads', 'nonconf_mapped_reads', 'total.reads', 'doublet.score', 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size'
    var: 'gene_id', 'gene_name'

In [87]:
adata.obs[['cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size']]

,cluster_id,cluster_label,subclass_label,class_label,cluster_color,size
barcode,,,,,,
AAACCCAAGCTTCATG-1L8TX_181211_01_G12,42,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334.0
AAACCCAAGGCGCTTC-1L8TX_181211_01_G12,NaN,NaN,NaN,NaN,NaN,NaN
AAACCCAAGGCTTAAA-1L8TX_181211_01_G12,NaN,NaN,NaN,NaN,NaN,NaN
AAACCCAAGTGAGGTC-1L8TX_181211_01_G12,41,L5 IT Tcap_1,L5 IT,Glutamatergic,#5DDB65,7462.0
AAACCCACACCAGCCA-1L8TX_181211_01_G12,42,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334.0
...,...,...,...,...,...,...
TTTGTTGTCATTGCGA-12L8TX_190430_01_G08,41,L5 IT Tcap_1,L5 IT,Glutamatergic,#5DDB65,7462.0
TTTGTTGTCCCAACTC-12L8TX_190430_01_G08,43,L5 IT S100b,L5 IT,Glutamatergic,#00CF1E,4272.0
TTTGTTGTCCGACAGC-12L8TX_190430_01_G08,NaN,NaN,NaN,NaN,NaN,NaN


# Preprocessing

In [88]:
sc.tl.pca(adata, svd_solver='arpack')

In [89]:
adata

AnnData object with n_obs × n_vars = 176584 × 31053
    obs: 'aggr_num', 'umi.counts', 'gene.counts', 'library_id', 'tube_barcode', 'Seq_batch', 'Region', 'Lib_type', 'Gender', 'Donor', 'Amp_Name', 'Amp_Date', 'Amp_PCR_cyles', 'Lib_Name', 'Lib_Date', 'Replicate_Lib', 'Lib_PCR_cycles', 'Lib_PassFail', 'Cell_Capture', 'Lib_Cells', 'Mean_Reads_perCell', 'Median_Genes_perCell', 'Median_UMI_perCell', 'Saturation', 'Live_percent', 'Total_Cells', 'Live_Cells', 'method', 'exp_component_name', 'mapped_reads', 'unmapped_reads', 'nonconf_mapped_reads', 'total.reads', 'doublet.score', 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size'
    var: 'gene_id', 'gene_name'
    uns: 'pca'
    obsm: 'X_pca'
    varm: 'PCs'

# Save

In [ ]:
adata.write_loom("data/processed/scRNAseq_10x_v3_AIBS.loom", write_obsm_varm=True)